In [1]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np

url = 'https://upload.wikimedia.org/wikipedia/commons/f/f2/Brachyscome_iberidifolia_Toucan_Tango_2zz.jpg'

# Wikimedia access requires a user agent
headers = {
    'User-Agent': 'DataDiary/0.1 (jared@datadiary.dev)'
}
response = requests.get(url, headers=headers)
image = Image.open(BytesIO(response.content))
image = image.convert('RGB')

image_array = np.array(image)
height = image_array.shape[0]
width = image_array.shape[1]
image_array_flat = image_array.flatten().reshape((
    height * width,
    3
))

view = image_array_flat.view((
    np.void,
    image_array_flat.dtype.itemsize * image_array_flat.shape[1]
))
num_colors = len(np.unique(view))

print(image_array_flat.shape, num_colors)

(665000, 3) 141862


In [2]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
labels = kmeans.fit_predict(image_array_flat)
centroids = np.round(kmeans.cluster_centers_).astype('uint8')

quantized_image_array_flat = centroids[labels]
quantized_image_array = quantized_image_array_flat.reshape(image_array.shape)

quantized_image = Image.fromarray(quantized_image_array, 'RGB')

quantized_image.save('Brachyscome_iberidifolia_Toucan_Tango_2zz_KMEANS3.png', 'PNG')
quantized_image.show()

In [23]:
import plotly.graph_objects as go

# sample 10k pixels to appear in the visual
image_array_flat_sample = image_array_flat[np.random.choice(image_array_flat.shape[0], 10_000)]

# plot pixels
fig = go.Figure(data=[go.Scatter3d(
    x=image_array_flat_sample[:, 0],
    y=image_array_flat_sample[:, 1],
    z=image_array_flat_sample[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=[f'rgb({pixel[0]}, {pixel[1]}, {pixel[2]})' for pixel in image_array_flat_sample],
        opacity=0.05,
        line=dict(width=0)
    ),
    hovertemplate = '<b>R</b>: %{x}<br><b>G</b>: %{y}<br><b>B</b>: %{z}<extra></extra>'
)])

# plot centroids
fig.add_trace(go.Scatter3d(
    x=centroids[:, 0],
    y=centroids[:, 1],
    z=centroids[:, 2],
    mode='markers',
    marker=dict(
        size=12,
        color=[f'rgb({centroid[0]}, {centroid[1]}, {centroid[2]})' for centroid in centroids],
        opacity=1.0,
        line=dict(width=0)
    ),
    hovertemplate = '<b>R</b>: %{x}<br><b>G</b>: %{y}<br><b>B</b>: %{z}<extra></extra>'
))

fig.update_layout(
    scene=dict(
        xaxis=dict(title='R'),
        yaxis=dict(title='G'),
        zaxis=dict(title='B')
    )
)

fig.show()

with open('centroids_in_color_space.json', 'w') as outfile:
    outfile.write(fig.to_json())